# Interpolation

## Common functions

In [2]:
import sys

# x_arr must be monotonically increasing array
def find_index(x_arr, x):
    if (x < x_arr[0] or x > x_arr[-1]):
        print("x is not inside x_arr, so exit..")
        sys.exit(0)
    for j in range(len(x_arr)):
        if (x_arr[j] >= x):
            i = j
            break
    return i

## Linear interpolation

In [3]:
def linear_interpolation(x_arr, y_arr, x):
    i = find_index(x_arr, x)
    a = (y_arr[i] - y_arr[i - 1]) / (x_arr[i] - x_arr[i - 1])
    b = y_arr[i - 1] - a * x_arr[i - 1]
    return a * x + b

## Square interpolation

In [4]:
import numpy as np

def square_interpolation(x_arr, y_arr, x):
    i = find_index(x_arr, x)
    x1, x2, x3 = x_arr[(slice(i - 1, i + 2) if i < len(x_arr) - 1 else slice(i - 3, i))]
    y1, y2, y3 = y_arr[(slice(i - 1, i + 2) if i < len(y_arr) - 1 else slice(i - 3, i))]
    a, b, c = np.linalg.solve(np.array([[x1**2, x1, 1], [x2**2, x2, 1], [x3**2, x3, 1]]), 
                              np.array([y1, y2, y3]))
    return a * x**2 + b * x + c

## Lagrange polynomial

In [5]:
def lagrange_interpolation(x_arr, y_arr, x):
    L = 0.0
    for i in range (0, len(x_arr)):
        l = 1.0
        for j in range (0, len(x_arr)):
            if i != j:
                l *= (x - x_arr[j]) / (x_arr[i] - x_arr[j])
        L += l * y_arr[i]
    return L

## For equidistant nodes

### Newton forward interpolation

In [6]:
import math

# Calculates n-th order finite difference
def calc_diff(y_arr, n, i):
    if n != 0:
        return calc_diff(y_arr, n - 1, i + 1) - calc_diff(y_arr, n - 1, i)
    else:
        return y_arr[i + 1] - y_arr[i]

def newton_forward_interpolation(x_arr, y_arr, x):
    i = find_index(x_arr, x) - 1
    n = len(x_arr) - 1
    h = x_arr[1] - x_arr[0]
    t = (x - x_arr[i]) / h
    return y_arr[i] + sum(np.prod([t - j for j in range(k)]) / math.factorial(k) * calc_diff(y_arr, k - 1, i) 
                          for k in range(1, n - i + 1))

### Newton backward interpolation

In [7]:
def newton_backward_interpolation(x_arr, y_arr, x):
    n = len(x_arr) - 1
    h = x_arr[1] - x_arr[0]
    t = (x - x_arr[n]) / h
    return y_arr[n] + sum(np.prod([t + j for j in range(k)]) / math.factorial(k) * calc_diff(y_arr, k - 1, n - k) 
                          for k in range(1, n + 1))

## For non-equidistant nodes

In [8]:
def calc_div_diff_1(x_arr, y_arr, i0, i1):
    return (y_arr[i1] - y_arr[i0]) / (x_arr[i1] - x_arr[i0])

def calc_div_diff_2(x_arr, y_arr, i0, i1, i2):
    return (calc_div_diff_1(x_arr, y_arr, i1, i2) - calc_div_diff_1(x_arr, y_arr, i0, i1)) / \
        (x_arr[i2] - x_arr[i0])

def newton_diff_interpolation(x_arr, y_arr, x):
    if len(x_arr) < 4:
        return None
    i0 = 0
    for i in range(0, len(x_arr)):
        if i == len(x_arr) - 4 or len(x_arr) - 4 <= 0:
            i0 = len(x_arr) - 4
            break
        if i == 0 and x <= x_arr[0]:
            break
        if i != 0 and x <= x_arr[i] and x > x_arr[i + 1]:
            i0 = i
            break
    i1 = i0 + 1; i2 = i1 + 1
    fir = y_arr[i0] + (x - x_arr[i0]) * calc_div_diff_1(x_arr, y_arr, i0, i1) + \
        (x - x_arr[i0]) * (x - x_arr[i1]) * calc_div_diff_2(x_arr, y_arr, i0, i1, i2)
    i0 += 1; i1 += 1; i2 += 1
    sec = y_arr[i0] + (x - x_arr[i0]) * calc_div_diff_1(x_arr, y_arr, i0, i1) + \
        (x - x_arr[i0]) * (x - x_arr[i1]) * calc_div_diff_2(x_arr, y_arr, i0, i1, i2)
    return (fir + sec) / 2

## Userspace

In [9]:
# You can customise file names right in code below
try:
    f = open("lagrange_input", "r")
    x_arr_lagr = [float(x) for x in f.readline().split()]
    y_arr_lagr = [float(y) for y in f.readline().split()]
except:
    print("File not found or have unsupported format..")
    sys.exit(0)
    
try:
    f = open("newton_input", "r")
    x_arr_newt = [float(x) for x in f.readline().split()]
    y_arr_newt = [float(y) for y in f.readline().split()]
except:
    print("File not found or have unsupported format..")
    sys.exit(0)

# debug mode on
print("Xs for Lagrange:", x_arr_lagr)
print("Ys for Lagrange:", y_arr_lagr, end="\n\n")
print("Xs for Newton  :", x_arr_newt)
print("Ys for Newton  :", y_arr_newt, end="\n\n")

def take_and_validate(array):
    while(True):
        try:
            value = float(input())
            if value < array[0] or value > array[-1]:
                print("Entered value is not inside array, try again: ")
            else:
                return value
        except:
            print("Entered value is not float, try again: ")
            

print("Enter value for linear, square and Lagrange interpolations: ", end="")
x_lsl = take_and_validate(x_arr_lagr)

print("Enter value for equidistant nodes Newton interpolation: ", end="")
x_newt = take_and_validate(x_arr_newt)

print("Enter value for non-equidistant nodes Newton interpolation: ", end="")
x_newt_non_eq = take_and_validate(x_arr_lagr)
print()

y_lin = linear_interpolation(x_arr_lagr, y_arr_lagr, x_lsl)
print(f"Linear interpolation for x = {x_lsl} - y = {round(y_lin, 4)}")

y_sqr = square_interpolation(x_arr_lagr, y_arr_lagr, x_lsl)
print(f"Square interpolation for x = {x_lsl} - y = {round(y_sqr, 4)}")

y_lagr = lagrange_interpolation(x_arr_lagr, y_arr_lagr, x_lsl)
print(f"Lagrange interpolation for x = {x_lsl} - y = {round(y_lagr, 6)}")

y_newt_forw = newton_forward_interpolation(x_arr_newt, y_arr_newt, x_newt)
print(f"Equidistant nodes Newton forward interpolation for x = {x_newt} - y = {round(y_newt_forw, 6)}")

y_newt_back = newton_backward_interpolation(x_arr_newt, y_arr_newt, x_newt)
print(f"Equidistant nodes Newton backward interpolation for x = {x_newt} - y = {round(y_newt_back, 6)}")

y_newt_non_eq = newton_diff_interpolation(x_arr_lagr, y_arr_lagr, x_newt_non_eq)
print(f"Non-equidistant nodes Newton interpolation for x = {x_newt_non_eq} - y = {round(y_newt_non_eq, 4)}")

# x1 = 0.720 # for linear, square and Lagrange
# x2 = 1.153 # for equidistant nodes Newton
# x3 = 1.852 # for equidistant nodes Newton
# x4 = 0.750 # for non-equidistant nodes Newtonе

Xs for Lagrange: [0.698, 0.706, 0.714, 0.727, 0.736, 0.747, 0.76]
Ys for Lagrange: [2.2234, 2.2438, 2.2644, 2.2984, 2.3222, 2.3516, 2.3867]

Xs for Newton  : [1.1, 1.25, 1.4, 1.55, 1.7, 1.85, 2.0]
Ys for Newton  : [0.2234, 1.2438, 2.2644, 3.2984, 4.3222, 5.3516, 6.3867]

Enter value for linear, square and Lagrange interpolations: 0.720
Enter value for equidistant nodes Newton interpolation: 1.153
Enter value for non-equidistant nodes Newton interpolation: 0.750

Linear interpolation for x = 0.72 - y = 2.2801
Square interpolation for x = 0.72 - y = 2.28
Lagrange interpolation for x = 0.72 - y = 2.280025
Equidistant nodes Newton forward interpolation for x = 1.153 - y = 0.591656
Equidistant nodes Newton backward interpolation for x = 1.153 - y = 0.591656
Non-equidistant nodes Newton interpolation for x = 0.75 - y = 2.3597
